This notebook uses the model created in pretrain any model notebook.

1. Pretrain Roberta Model: https://www.kaggle.com/maunish/clrp-pytorch-roberta-pretrain
2. Finetune Roberta Model: this notebook, <br/>
   Finetune Roberta Model TPU: https://www.kaggle.com/maunish/clrp-pytorch-roberta-finetune-tpu
3. Inference Notebook: https://www.kaggle.com/maunish/clrp-pytorch-roberta-inference
4. Roberta + SVM: https://www.kaggle.com/maunish/clrp-roberta-svm

In [ ]:
!nvidia-smi

Mon Jun 28 15:26:23 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8    27W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install -q kaggle
!mkdir /root/.kaggle
!cp /content/drive/MyDrive/Colab\ Notebooks/kaggle.json /root/.kaggle/

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# データセット入れる箱
# !mkdir /content/drive/MyDrive/CommonLit/input/commonlitreadabilityprize/clrprobertalarge

In [ ]:
!kaggle datasets download -d  takeshikobayashi/clrprobertalarge
!unzip -n clrprobertalarge -d /content/drive/MyDrive/CommonLit/input/commonlitreadabilityprize/clrprobertalarge
# !unzip -n clrprobertalarge -d clrprobertalarge

# !kaggle datasets download -d  takeshikobayashi/keras-nn-with-features-neutralization-ver18
# !unzip -n keras-nn-with-features-neutralization-ver18 -d keras-nn-with-features-neutralization-ver18

clrprobertalarge.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  clrprobertalarge.zip
  inflating: /content/drive/MyDrive/CommonLit/input/commonlitreadabilityprize/clrprobertalarge/clrp_robertalarge_base/config.json  
  inflating: /content/drive/MyDrive/CommonLit/input/commonlitreadabilityprize/clrprobertalarge/clrp_robertalarge_base/merges.txt  
  inflating: /content/drive/MyDrive/CommonLit/input/commonlitreadabilityprize/clrprobertalarge/clrp_robertalarge_base/pytorch_model.bin  
  inflating: /content/drive/MyDrive/CommonLit/input/commonlitreadabilityprize/clrprobertalarge/clrp_robertalarge_base/special_tokens_map.json  
  inflating: /content/drive/MyDrive/CommonLit/input/commonlitreadabilityprize/clrprobertalarge/clrp_robertalarge_base/tokenizer_config.json  
  inflating: /content/drive/MyDrive/CommonLit/input/commonlitreadabilityprize/clrprobertalarge/clrp_robertalarge_base/training_args.bin  
  inflating: /content/drive/MyDrive/Comm

In [ ]:
!pip install transformers
!pip install colorama 

In [ ]:
import os
import gc
import sys
import math

import time
import tqdm
import random
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold

import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from transformers import (AutoModel,AutoTokenizer,get_cosine_schedule_with_warmup)
from transformers import PreTrainedModel, RobertaTokenizerFast, RobertaConfig, RobertaModel, AdamW,BertConfig

from colorama import Fore, Back, Style
r_ = Fore.RED
b_ = Fore.BLUE
c_ = Fore.CYAN
g_ = Fore.GREEN
y_ = Fore.YELLOW
m_ = Fore.MAGENTA
sr_ = Style.RESET_ALL

In [ ]:
# train_data = pd.read_csv('../input/commonlitreadabilityprize/train.csv')
train_data = pd.read_csv('/content/drive/MyDrive/CommonLit/input/commonlitreadabilityprize/train.csv')
test_data = pd.read_csv('/content/drive/MyDrive/CommonLit/input/commonlitreadabilityprize/test.csv')
sample = pd.read_csv('/content/drive/MyDrive/CommonLit/input/commonlitreadabilityprize/sample_submission.csv')

num_bins = int(np.floor(1 + np.log2(len(train_data))))
train_data.loc[:,'bins'] = pd.cut(train_data['target'],bins=num_bins,labels=False)
# train_data['excerpt'] = train_data['excerpt'].apply(lambda x: x.replace('\n',''))

bins = train_data.bins.to_numpy()
target = train_data.target.to_numpy()

def rmse_score(y_true,y_pred):
    return np.sqrt(mean_squared_error(y_true,y_pred))

In [ ]:
config = {
    'lr': 5e-5,#2e-5
    'wd':0.01,
    'batch_size':8,#16
    'valid_step':5,#10
    'max_len':256,
    'epochs':3,
    'nfolds':5,
    'seed':42,
}

for i in range(config['nfolds']):
    os.makedirs(f'model{i}',exist_ok=True)

def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONASSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(seed=config['seed'])

train_data['Fold'] = -1
kfold = StratifiedKFold(n_splits=config['nfolds'],shuffle=True,random_state=config['seed'])
for k , (train_idx,valid_idx) in enumerate(kfold.split(X=train_data,y=bins)):
    train_data.loc[valid_idx,'Fold'] = k

In [ ]:
train_data.head()

,id,url_legal,license,excerpt,target,standard_error,bins,Fold
0,c12129c31,NaN,NaN,When the young people returned to the ballroom...,-0.340259,0.464009,7,0
1,85aa80a4c,NaN,NaN,"All through dinner time, Mrs. Fayre was somewh...",-0.315372,0.480805,7,2
2,b69ac6792,NaN,NaN,"As Roger had predicted, the snow departed as q...",-0.580118,0.476676,6,3
3,dd1000b26,NaN,NaN,And outside before the palace a great garden w...,-1.054013,0.450007,5,2
4,37c1b32fb,NaN,NaN,Once upon a time there were Three Bears who li...,0.247197,0.510845,8,1


In [ ]:
class CLRPDataset(Dataset):
    def __init__(self,df,tokenizer,max_len=128):
        self.excerpt = df['excerpt'].to_numpy()
        self.targets = df['target'].to_numpy()
        self.max_len = max_len
        self.tokenizer = tokenizer
    
    def __getitem__(self,idx):
        encode = self.tokenizer(self.excerpt[idx],
                                return_tensors='pt',
                                max_length=self.max_len,
                                padding='max_length',
                                truncation=True)
        
        target = torch.tensor(self.targets[idx],dtype=torch.float) 
        return encode, target
    
    def __len__(self):
        return len(self.excerpt)

In [ ]:
class AttentionHead(nn.Module):
    def __init__(self, in_features, hidden_dim, num_targets):
        super().__init__()
        self.in_features = in_features
        self.middle_features = hidden_dim
        self.W = nn.Linear(in_features, hidden_dim)
        self.V = nn.Linear(hidden_dim, 1)
        self.out_features = hidden_dim

    def forward(self, features):
        att = torch.tanh(self.W(features))
        score = self.V(att)
        attention_weights = torch.softmax(score, dim=1)
        context_vector = attention_weights * features
        context_vector = torch.sum(context_vector, dim=1)

        return context_vector

In [ ]:
class Model(nn.Module):
    def __init__(self,path):
        super(Model,self).__init__()
        self.roberta = AutoModel.from_pretrained(path,output_hidden_states=True)  
        self.head = AttentionHead(768,768,1)
        self.dropout = nn.Dropout(0.1)
#         self.linear = nn.Linear(768,1)
#         self.linear=nn.Linear(768,1)#Roberta
        self.linear = nn.Linear(1024, 1)#Roberta-large


    def forward(self,**xb):

########## original ##########        
#         x = self.roberta(**xb)[0]
#         x = self.head(x)
#         x = self.dropout(x)
#         x = self.linear(x)
#         return x
        
        output1 = self.roberta(**xb)
        output1 = output1.hidden_states
        output1 = output1[-1]
        output1 = self.dropout(output1)
        output1 = torch.mean(output1, 1, True)
        final_output = self.linear(output1)
        final_outputs = final_output.squeeze(-1).squeeze(-1)
        
        return final_outputs
        
        
#         output1 = self.roberta(input_ids=ids, attention_mask=mask)
#         output1 = output1.hidden_states
#         output1 = output1[-1]
#         output1 = self.dropout(output1)
#         output1 = torch.mean(output1, 1, True)
# #         print(output1.size())
#         final_output = self.linear(output1)
#         final_outputs = final_output.squeeze(-1).squeeze(-1)
        
#         return x

In [ ]:
def run(fold,verbose=True):
    
    def loss_fn(outputs,targets):
        outputs = outputs.view(-1)
        targets = targets.view(-1)
        return torch.sqrt(nn.MSELoss()(outputs,targets))
    
    def train_and_evaluate_loop(train_loader,valid_loader,model,loss_fn, device,optimizer,epoch,fold,best_loss,valid_step=10,lr_scheduler=None):
        train_loss = 0
        for i, (inputs1,targets1) in enumerate(train_loader):
            model.train()
            optimizer.zero_grad()
#             print(inputs1.shape)
            inputs1 = {key:val.reshape(val.shape[0],-1).to(device) for key,val in inputs1.items()}
            targets1 = targets1.to(device)
            outputs1 = model(**inputs1)
            loss1 = loss_fn(outputs1,targets1)
            loss1.backward()
            optimizer.step()
            
            train_loss += loss1.item()
            
            if lr_scheduler:
                lr_scheduler.step()
            
            #evaluating for every valid_step
            if (i % valid_step == 0) or ((i + 1) == len(train_loader)):
                model.eval()
                valid_loss = 0
                with torch.no_grad():
                    for j, (inputs2,targets2) in enumerate(valid_loader):
                        inputs2 = {key:val.reshape(val.shape[0],-1).to(device) for key,val in inputs2.items()}
                        targets2 = targets2.to(device)
                        outputs2 = model(**inputs2)
                        loss2 = loss_fn(outputs2,targets2)
                        valid_loss += loss2.item()
                     
                    valid_loss /= len(valid_loader)
                    if valid_loss <= best_loss:
                        if verbose:
                            print(f"epoch:{epoch} | Train Loss:{train_loss/(i+1)} | Validation loss:{valid_loss}")
                            print(f"{g_}Validation loss Decreased from {best_loss} to {valid_loss}{sr_}")

                        best_loss = valid_loss
                        torch.save(model.state_dict(),f'./model{fold}/model{fold}_ver1.bin')
                        torch.save(model.state_dict(),f'./model{fold}/model{fold}_ver1.h5')
                        tokenizer.save_pretrained(f'./model{fold}')
                        
        return best_loss
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
                
    x_train,x_valid = train_data.query(f"Fold != {fold}"),train_data.query(f"Fold == {fold}")
    print("test")
#     MODEL_PATH = f'../input/clrp-roberta-base/clrp_roberta_base'
    MODEL_PATH = (f'/content/drive/MyDrive/CommonLit/input/commonlitreadabilityprize/clrprobertalarge/clrp_robertalarge_base')
    tokenizer = AutoTokenizer.from_pretrained('roberta-large')
#     tokenizer = AutoTokenizer.from_pretrained('roberta-base')
    model_config = RobertaConfig.from_pretrained('roberta-large')
    model_config.output_hidden_states = True
    model = Model(MODEL_PATH)
    model.to(device)
    print("test2")
    train_ds = CLRPDataset(x_train,tokenizer,config['max_len'])
    train_dl = DataLoader(train_ds,
                        batch_size = config["batch_size"],
                        shuffle=True,
                        num_workers = 4,
                        pin_memory=True,
                        drop_last=False)
    print("test3")
    valid_ds = CLRPDataset(x_valid,tokenizer,config['max_len'])
    valid_dl = DataLoader(valid_ds,
                        batch_size = config["batch_size"],
                        shuffle=False,
                        num_workers = 4,
                        pin_memory=True,
                        drop_last=False)

    optimizer = optim.AdamW(model.parameters(),lr=config['lr'],weight_decay=config['wd'])
    lr_scheduler = get_cosine_schedule_with_warmup(optimizer,num_warmup_steps=0,num_training_steps= 10 * len(train_dl))
#     lr_scheduler = None

    print(f"Fold: {fold}")
    best_loss = 9999
    for epoch in range(config["epochs"]):
        print(f"Epoch Started:{epoch}")
        best_loss = train_and_evaluate_loop(train_dl,valid_dl,model,loss_fn,
                                            device,optimizer,epoch,fold,best_loss,
                                            valid_step=config['valid_step'],lr_scheduler=lr_scheduler)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('roberta-large')
model_config = RobertaConfig.from_pretrained('roberta-large')
model_config.output_hidden_states = True
model = Model('/content/drive/MyDrive/CommonLit/input/commonlitreadabilityprize/clrprobertalarge/clrp_robertalarge_base')
model.to(device)

Some weights of the model checkpoint at /content/drive/MyDrive/CommonLit/input/commonlitreadabilityprize/clrprobertalarge/clrp_robertalarge_base were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.decoder.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at /content/drive/MyDrive/CommonLit/input/commonlitreadabilityprize/clrprobertalarge/clrp_robertalarge_ba

NameError: ignored

In [ ]:
for f in range(config['nfolds']):
    run(f)

test


Some weights of the model checkpoint at /content/drive/MyDrive/CommonLit/input/commonlitreadabilityprize/clrprobertalarge/clrp_robertalarge_base were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.decoder.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at /content/drive/MyDrive/CommonLit/input/commonlitreadabilityprize/clrprobertalarge/clrp_robertalarge_ba

test2
test3
Fold: 0
Epoch Started:0


KeyboardInterrupt: ignored